<a href="https://colab.research.google.com/github/LuisAngelOlveraOlvera/Grokking-Deep-Learning/blob/main/10_intro_to_convolutional_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.datasets import mnist

# Configuración inicial  y carga de datos
np.random.seed(1)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
images, labels = (x_train[0:1000].reshape(1000, 28*28)/255, y_train[0:1000])
test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = y_test

# Conversión de etiquetas a one-hot encoding
one_hot_labels = np.zeros((len(labels), 10))

for i, l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels = one_hot_labels

print("labels: ", labels)

test_one_hot_labels = np.zeros((len(test_labels), 10))

for i, l in enumerate(test_labels):
  test_one_hot_labels[i][l] = 1
test_labels = test_one_hot_labels

print("test_labels: ", test_labels)

# Funciones de activación
def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

# Inicialización de parámetros
alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128
input_rows, input_cols = (28, 28)
kernel_rows, kernel_cols = (3, 3)
num_kernels = 16
hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels
print("hidden_size: ", hidden_size)

labels:  [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
test_labels:  [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
hidden_size:  10000


In [ ]:
kernels = 0.02 * np.random.random((kernel_rows * kernel_cols, num_kernels))- 0.01
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels))-0.1
print("kernels: ", kernels)
print("weights_1_2: ", weights_1_2)

kernels:  [[ 1.70328231e-03 -5.97770204e-03 -1.72637459e-03  7.66328068e-03
   3.02881408e-03  8.56675338e-03  5.51296256e-04  3.26438096e-03
   4.90987642e-04  7.06176896e-03 -5.62288802e-03 -2.75093130e-03
   2.78062110e-03 -6.44540395e-03 -7.59203775e-03  4.98202336e-03]
 [ 3.41203030e-03 -8.69349186e-03  1.01748480e-03  2.05430893e-03
  -6.95127390e-03 -4.41626201e-03 -1.62584550e-03  6.42637849e-03
  -5.60240168e-03  2.71610574e-04 -8.04507054e-03  6.20940038e-03
   3.25843127e-03 -1.78213333e-03  6.01667078e-03  6.30671014e-03]
 [-8.35596746e-03 -5.04125968e-03 -3.67738299e-03 -9.18998871e-03
  -8.00852519e-03 -9.48393207e-03 -8.37650939e-03  4.37158386e-03
   2.43512431e-03 -8.03564237e-03 -5.38022960e-03  4.67030246e-04
  -2.71828310e-03  3.09839204e-03 -9.17337467e-03  5.64183692e-03]
 [-4.82516555e-03 -7.70515482e-03 -9.57758920e-03 -2.57027131e-03
   3.40429098e-03  7.38807194e-03  8.30585381e-03 -3.85651706e-03
   6.85507543e-03 -3.49562910e-04  8.08055344e-03 -3.33690190e-

In [ ]:
# función para extraer secciones de la imagen
def get_image_section(layer, row_from, row_to, col_from, col_to):
  sub_section = layer[:, row_from:row_to, col_from:col_to]
  return sub_section.reshape(-1,1, row_to - row_from, col_to - col_from)

# entrenamiento del modelo
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = (i * batch_size, (i + 1) * batch_size)
        layer_0 = images[batch_start:batch_end].reshape(batch_size, input_rows, input_cols)

        sects = []
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0, row_start, row_start + kernel_rows, col_start, col_start + kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))

        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask *2

        layer_2 = softmax(np.dot(layer_1, weights_1_2))

        for k in range(batch_size):
          labelset = labels[batch_start + k:batch_start + k+1]
          correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labelset))

        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update

# Prueba del modelo
    # Prueba del modelo
    test_correct_cnt = 0
    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1].reshape(1, 28, 28)

        sects = []
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0, row_start, row_start + kernel_rows, col_start, col_start + kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = np.dot(layer_1, weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

    if j % 1 == 0:
        print("\nIteration:", j,
              "Test Accuracy:", test_correct_cnt / float(len(test_images)),
              "Train Accuracy:", correct_cnt / float(len(images)))








Iteration: 0 Test Accuracy: 0.8738 Train Accuracy: 0.802

Iteration: 1 Test Accuracy: 0.8773 Train Accuracy: 0.824

Iteration: 2 Test Accuracy: 0.8793 Train Accuracy: 0.807

Iteration: 3 Test Accuracy: 0.8772 Train Accuracy: 0.812

Iteration: 4 Test Accuracy: 0.8787 Train Accuracy: 0.817

Iteration: 5 Test Accuracy: 0.8757 Train Accuracy: 0.812

Iteration: 6 Test Accuracy: 0.8771 Train Accuracy: 0.826

Iteration: 7 Test Accuracy: 0.8766 Train Accuracy: 0.822

Iteration: 8 Test Accuracy: 0.8777 Train Accuracy: 0.806

Iteration: 9 Test Accuracy: 0.878 Train Accuracy: 0.812

Iteration: 10 Test Accuracy: 0.8777 Train Accuracy: 0.821

Iteration: 11 Test Accuracy: 0.8782 Train Accuracy: 0.806

Iteration: 12 Test Accuracy: 0.8784 Train Accuracy: 0.813

Iteration: 13 Test Accuracy: 0.8784 Train Accuracy: 0.816

Iteration: 14 Test Accuracy: 0.8773 Train Accuracy: 0.826

Iteration: 15 Test Accuracy: 0.879 Train Accuracy: 0.815

Iteration: 16 Test Accuracy: 0.8774 Train Accuracy: 0.811

Iteratio

IndentationError: unexpected indent (<ipython-input-20-65760a705e61>, line 3)